In [ ]:
import pandas as pd
import requests
pd.set_option('display.max_columns', None)
import numpy as np
from nba_api.stats.endpoints import LeagueDashPlayerStats
from nba_api.stats.endpoints import commonteamroster
import time


### RetrieveNBA Api Player Level Data using NBA API

In [ ]:
def fetch_nba_season_stats(season):
    """
    Fetches NBA player statistics for both the regular season and playoffs for a given season.

    Parameters:
    - season (str): The NBA season in "YYYY-YY" format (e.g., "2023-24").

    Returns:
    - df_regular (DataFrame): Regular season player stats.
    - df_playoffs (DataFrame): Playoff player stats.
    """

    # Fetch regular season stats
    player_stats_regular = LeagueDashPlayerStats(season=season, season_type_all_star="Regular Season")
    df_regular = player_stats_regular.get_data_frames()[0]
    df_regular["Season Type"] = "Regular Season"

    # Fetch playoff stats
    player_stats_playoffs = LeagueDashPlayerStats(season=season, season_type_all_star="Playoffs")
    df_playoffs = player_stats_playoffs.get_data_frames()[0]
    df_playoffs["Season Type"] = "Playoffs"

    return df_regular, df_playoffs

from nba_api.stats.endpoints import LeagueDashPlayerStats

def fetch_nba_season_advanced_stats(season):
    """
    Fetches NBA player advanced statistics for both the regular season and playoffs for a given season.

    Parameters:
    - season (str): The NBA season in "YYYY-YY" format (e.g., "2023-24").

    Returns:
    - df_regular_adv (DataFrame): Regular season player advanced stats.
    - df_playoffs_adv (DataFrame): Playoff player advanced stats.
    """
    # Fetch regular season advanced stats
    player_stats_regular_adv = LeagueDashPlayerStats(
        season=season,
        season_type_all_star="Regular Season",
        measure_type_detailed_defense="Advanced"
    )
    df_regular_adv = player_stats_regular_adv.get_data_frames()[0]
    df_regular_adv["Season Type"] = "Regular Season"

    # Fetch playoff advanced stats
    player_stats_playoffs_adv = LeagueDashPlayerStats(
        season=season,
        season_type_all_star="Playoffs",
        measure_type_detailed_defense="Advanced"
    )
    df_playoffs_adv = player_stats_playoffs_adv.get_data_frames()[0]
    df_playoffs_adv["Season Type"] = "Playoffs"

    return df_regular_adv, df_playoffs_adv

In [ ]:
# Basic Stats
season_years = {
    2023: "2023-24",
    2022: "2022-23",
    2021: "2021-22",
    2020: "2020-21",
    2019: "2019-20"
}

regular_season_dfs = []

for start_year, season_str in season_years.items():
    df_regular, df_playoffs = fetch_nba_season_stats(season_str)
    
    # Add 'season' column (use start year)
    df_regular['Season'] = start_year
    
    # Append to list for later merging
    regular_season_dfs.append(df_regular)

# Merge all regular season DataFrames
df_regular_all = pd.concat(regular_season_dfs, ignore_index=True)

In [ ]:
# Advance Stats
advanced_season_dfs = []

for start_year, season_str in season_years.items():
    df_regular_adv, df_playoffs_adv = fetch_nba_season_advanced_stats(season_str)
    
    # Add 'Season' column
    df_regular_adv['Season'] = start_year
    
    # Append to list
    advanced_season_dfs.append(df_regular_adv)

# Merge all regular season advanced DataFrames
df_advanced_all = pd.concat(advanced_season_dfs, ignore_index=True)

# Merge on shared keys: PLAYER_ID, TEAM_ID, and Season
df_regular_all = pd.merge(
    df_regular_all,
    df_advanced_all,
    on=["PLAYER_ID", "TEAM_ID", "Season"],
    suffixes=('', '_adv'),
    how='left'
)

In [ ]:
df_regular_all.head(1)

### Retrieve Player Physical Stats

In [ ]:
# season = "2023-24"

# # Retrieve roster data 
# roster_response = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
# roster_df = roster_response.get_data_frames()[0]

# roster_df.head(1)

In [ ]:
roster_data = []

# Loop over each season in our mapping
for start_year, season_str in season_years.items():
    teams_in_season = df_regular_all[df_regular_all['Season'] == start_year]['TEAM_ID'].unique()
    
    for team_id in teams_in_season:
        try:
            roster_response = commonteamroster.CommonTeamRoster(team_id=team_id, season=season_str)
            roster_df = roster_response.get_data_frames()[0]
            
            required_cols = ["PLAYER_ID", "POSITION", "HEIGHT", "WEIGHT", "EXP"]
            if all(col in roster_df.columns for col in required_cols):
                tmp = roster_df[required_cols].copy()
                tmp["TEAM_ID"] = team_id
                tmp["Season"] = start_year
                roster_data.append(tmp)
            else:
                print(f"Team {team_id} for season {season_str} missing one or more required columns.")

            time.sleep(0.6)
        except Exception as e:
            print(f"Error retrieving roster for team {team_id} in season {season_str}: {e}")

# Combine roster data from all teams and seasons if any data was collected
if roster_data:
    roster_all = pd.concat(roster_data, ignore_index=True)
    df_regular_all = df_regular_all.merge(roster_all, on=["PLAYER_ID", "TEAM_ID", "Season"], how="left")
else:
    print("No roster data was retrieved.")

df_regular_all.head()

### Read in Injury Data

In [ ]:
il_df = pd.read_csv('IL Report 20 to 24 v2.csv', encoding='utf-8')
il_df['Season'] = il_df['Season'].replace(1999, 2019)
il_df

### Name Mapping

Streamline the names between two data sources.

In [ ]:
name_map_df = pd.read_excel('nba_regular_season_data v2.xlsx', sheet_name='Mapped Name Final')
name_map_df.columns

In [ ]:
name_mapping = dict(zip(name_map_df['Player (IL report)'], name_map_df['Mapped Name (NBA API)']))
il_df['mapped_name'] = il_df['Player'].map(name_mapping).fillna(il_df['Player'])
il_df

### Check for shared name, not shared names between injury and performance table

In [ ]:
il_df['mapped_name'] = il_df['mapped_name'].astype(str).str.strip()
df_regular_all['PLAYER_NAME'] = df_regular_all['PLAYER_NAME'].astype(str).str.strip()

injury_names = set(il_df['mapped_name'].unique())
performance_names = set(df_regular_all['PLAYER_NAME'].unique())

# Find names only in the injury table
only_in_injury = injury_names - performance_names

# Create a comparison DataFrame
comparison_df = pd.DataFrame({
    'player_name': list(injury_names.union(performance_names)),
    'in_injury_table': [name in injury_names for name in injury_names.union(performance_names)],
    'in_performance_table': [name in performance_names for name in injury_names.union(performance_names)],
    'only_in_injury_table': [name in only_in_injury for name in injury_names.union(performance_names)]
})

# Filter to just players only in the injury table, if needed
only_injury_df = comparison_df[comparison_df['only_in_injury_table']]
only_injury_df

In [ ]:
# Dropping players that doesn't have stats in NBA Api
exclude_players = list(only_injury_df['player_name'])
il_df = il_df[~il_df['mapped_name'].isin(exclude_players)]
il_df

### Merge Injury data with Performance data

In [ ]:
# il_df['Season_for_merge'] = il_df['Season'] - 1
df_regular_all = pd.merge(
    df_regular_all,
    il_df,
    left_on=['PLAYER_NAME', 'Season'],
    right_on=['mapped_name', 'Season'],
    how='left'
)

df_regular_all['Games Missed'] = df_regular_all['Games Missed'].fillna(0.0)
df_regular_all


### Impute Players Missing Roster Info

In [ ]:
# Step 1: Replace 'R' with 0 in EXP and convert to numeric
df_regular_all['EXP'] = df_regular_all['EXP'].replace('R', 0)
df_regular_all['EXP'] = pd.to_numeric(df_regular_all['EXP'], errors='coerce')

# Step 2: Create a helper table with known physical features
phys_ref = (
    df_regular_all[df_regular_all[['POSITION', 'HEIGHT', 'WEIGHT']].notna().all(axis=1)]
    .sort_values(by='Season')
    .groupby('PLAYER_ID')
    .apply(lambda x: x.set_index('Season')[['POSITION', 'HEIGHT', 'WEIGHT']])
)

# Step 3: Impute missing POSITION, HEIGHT, WEIGHT using closest known season
def impute_phys(row):
    if pd.notna(row['POSITION']) and pd.notna(row['HEIGHT']) and pd.notna(row['WEIGHT']):
        return row[['POSITION', 'HEIGHT', 'WEIGHT']]
    try:
        player_data = phys_ref.loc[row['PLAYER_ID']]
        closest_season = player_data.index.to_series().sub(row['Season']).abs().idxmin()
        return player_data.loc[closest_season]
    except:
        return pd.Series([row['POSITION'], row['HEIGHT'], row['WEIGHT']], index=['POSITION', 'HEIGHT', 'WEIGHT'])

df_regular_all[['POSITION', 'HEIGHT', 'WEIGHT']] = df_regular_all.apply(impute_phys, axis=1)

# Step 4: Impute EXP using the earliest known season for that player
rookie_exp_map = (
    df_regular_all[df_regular_all['EXP'].notna()]
    .groupby('PLAYER_ID')
    .apply(lambda x: x.loc[x['EXP'] == 0, 'Season'].min() if (x['EXP'] == 0).any() else x['Season'].min())
).to_dict()

def impute_exp(row):
    if pd.notna(row['EXP']):
        return row['EXP']
    rookie_season = rookie_exp_map.get(row['PLAYER_ID'])
    if rookie_season is not None:
        return max(0, row['Season'] - rookie_season)
    return np.nan

df_regular_all['EXP'] = df_regular_all.apply(impute_exp, axis=1)

In [ ]:
missing_player_df = pd.read_csv('miss_player.csv')
# Ensure clean player names
# Clean names just in case
df_regular_all["PLAYER_NAME"] = df_regular_all["PLAYER_NAME"].str.strip()
missing_player_df["Player Name"] = missing_player_df["Player Name"].str.strip()

# Filter trusted names
trusted_info = missing_player_df[
    missing_player_df["Player Name"].isin(df_regular_all["PLAYER_NAME"].unique())
].copy()

# Columns to impute
cols_to_impute = ["POSITION", "HEIGHT", "WEIGHT", "EXP"]

# Loop through rows in trusted_info
for _, info_row in trusted_info.iterrows():
    name = info_row["Player Name"]
    year_since = info_row["Year Since"]
    
    # Subset of df_regular_all that matches the player and has missing values
    mask = (
        (df_regular_all["PLAYER_NAME"] == name) &
        (
            df_regular_all["POSITION"].isna() |
            df_regular_all["HEIGHT"].isna() |
            df_regular_all["WEIGHT"].isna() |
            df_regular_all["EXP"].isna()
        )
    )
    
    # Fill only missing values
    for col, source_col in zip(["POSITION", "HEIGHT", "WEIGHT"], ["Position", "Height", "Weight"]):
        df_regular_all.loc[mask & df_regular_all[col].isna(), col] = info_row[source_col]
    
    # EXP: calculate from Season - Year Since, only if EXP is missing
    if not pd.isna(year_since):
        df_regular_all.loc[mask & df_regular_all["EXP"].isna(), "EXP"] = (
            df_regular_all.loc[mask & df_regular_all["EXP"].isna(), "Season"] - year_since
        )

# Final EXP check: clip negatives and set rookies to 0
df_regular_all["EXP"] = df_regular_all["EXP"].clip(lower=0)

### Additional Cleaning: Weight and Height

In [ ]:
import re

def height_to_inches(x):
    """
    Converts height formats like '6-5', '6'5"', or '6′5″' to inches.
    If already a float or int (i.e., already in inches), returns as-is.
    Idempotent — safe to apply multiple times.
    """
    # If already numeric, return it as is
    if isinstance(x, (int, float)) and not np.isnan(x):
        return x

    if pd.isna(x):
        return np.nan

    # Normalize common height formats
    x_str = str(x).strip()
    match = re.match(r"(\d+)[\'′-](\d+)", x_str)
    if match:
        feet, inches = map(int, match.groups())
        return feet * 12 + inches

    return np.nan


df['HEIGHT'] = df['HEIGHT'].apply(height_to_inches).astype('float')

In [ ]:
import numpy as np

def parse_weight(value):
    """
    Cleans up weight strings like '180 lbs' or '215 lbs' 
    and converts them to a float (pounds).
    """
    # If it's already missing, return NaN
    if pd.isna(value):
        return np.nan

    # Convert to lowercase string
    val_str = str(value).lower().strip()

    # Remove the substring 'lbs'
    val_str = val_str.replace('lbs', '').strip()

    # Attempt to convert to float
    try:
        return float(val_str)
    except ValueError:
        return np.nan

# Example usage on the entire 'WEIGHT' column
df['WEIGHT'] = df['WEIGHT'].apply(parse_weight)

In [ ]:
df.to_csv('data/nba_players_stats_2.csv', index=False, encoding='utf-8-sig')

# NBA Player Stats Column Definitions

| Column | Definition |
|--------|------------|
| `PLAYER_ID` | Unique identifier for the player. |
| `PLAYER_NAME` | Full name of the player. |
| `NICKNAME` | Nickname of the player (often empty or not used). |
| `TEAM_ID` | Unique identifier for the team. |
| `TEAM_ABBREVIATION` | Short abbreviation of the team name (e.g., LAL for Lakers). |
| `AGE` | Player’s age during the season. |
| `GP` | Games played by the player. |
| `W` | Number of games won when the player participated. |
| `L` | Number of games lost when the player participated. |
| `W_PCT` | Win percentage when the player was active (`W / GP`). |
| `MIN` | Average minutes played per game. |
| `FGM` | Field goals made per game. |
| `FGA` | Field goals attempted per game. |
| `FG_PCT` | Field goal percentage (`FGM / FGA`). |
| `FG3M` | Three-point field goals made per game. |
| `FG3A` | Three-point field goals attempted per game. |
| `FG3_PCT` | Three-point field goal percentage (`FG3M / FG3A`). |
| `FTM` | Free throws made per game. |
| `FTA` | Free throws attempted per game. |
| `FT_PCT` | Free throw percentage (`FTM / FTA`). |
| `OREB` | Offensive rebounds per game. |
| `DREB` | Defensive rebounds per game. |
| `REB` | Total rebounds per game (`OREB + DREB`). |
| `AST` | Assists per game. |
| `TOV` | Turnovers per game. |
| `STL` | Steals per game. |
| `BLK` | Blocks per game. |
| `BLKA` | Blocks against (how many times the player's shot was blocked). |
| `PF` | Personal fouls committed per game. |
| `PFD` | Personal fouls drawn per game (fouls suffered by the player). |
| `PTS` | Points per game. |
| `PLUS_MINUS` | Player’s net impact on the scoreboard (`team points - opponent points`) while they are on the court. |
| `NBA_FANTASY_PTS` | Fantasy points based on standard NBA fantasy scoring systems. |
| `DD2` | Number of double-doubles (when a player records 10+ in two statistical categories). |
| `TD3` | Number of triple-doubles (when a player records 10+ in three statistical categories). |
| `WNBA_FANTASY_PTS` | Likely a placeholder, as this dataset is NBA-only. |
| `GP_RANK` | Rank among all players for games played. |
| `W_RANK` | Rank among all players for wins. |
| `L_RANK` | Rank among all players for losses. |
| `W_PCT_RANK` | Rank among all players for win percentage. |
| `MIN_RANK` | Rank among all players for minutes played. |
| `FGM_RANK` | Rank among all players for field goals made. |
| `FGA_RANK` | Rank among all players for field goals attempted. |
| `FG_PCT_RANK` | Rank among all players for field goal percentage. |
| `FG3M_RANK` | Rank among all players for three-point field goals made. |
| `FG3A_RANK` | Rank among all players for three-point field goals attempted. |
| `FG3_PCT_RANK` | Rank among all players for three-point percentage. |
| `FTM_RANK` | Rank among all players for free throws made. |
| `FTA_RANK` | Rank among all players for free throws attempted. |
| `FT_PCT_RANK` | Rank among all players for free throw percentage. |
| `OREB_RANK` | Rank among all players for offensive rebounds. |
| `DREB_RANK` | Rank among all players for defensive rebounds. |
| `REB_RANK` | Rank among all players for total rebounds. |
| `AST_RANK` | Rank among all players for assists. |
| `TOV_RANK` | Rank among all players for turnovers. |
| `STL_RANK` | Rank among all players for steals. |
| `BLK_RANK` | Rank among all players for blocks. |
| `BLKA_RANK` | Rank among all players for blocked attempts. |
| `PF_RANK` | Rank among all players for personal fouls committed. |
| `PFD_RANK` | Rank among all players for personal fouls drawn. |
| `PTS_RANK` | Rank among all players for points scored. |
| `PLUS_MINUS_RANK` | Rank among all players for plus/minus impact. |
| `NBA_FANTASY_PTS_RANK` | Rank among all players for fantasy points. |
| `DD2_RANK` | Rank among all players for double-doubles. |
| `TD3_RANK` | Rank among all players for triple-doubles. |
| `WNBA_FANTASY_PTS_RANK` | Placeholder for WNBA stats (likely unused in this dataset). |
| `Season Type` | Type of season (`Regular Season`, `Playoffs`, etc.). |

## 📊 Undocumented NBA Stats Column Definitions

| Column | Definition |
|--------|------------|
| `E_OFF_RATING` | Estimated Offensive Rating: A version of offensive rating that estimates points produced per 100 possessions using adjusted metrics. |
| `OFF_RATING` | Offensive Rating: Points produced per 100 possessions while the player is on the court. |
| `sp_work_OFF_RATING` | A special/internal calculation of offensive rating used in SportsVu or internal tools—typically close to OFF_RATING. Often for testing/validation. |
| `E_DEF_RATING` | Estimated Defensive Rating: An estimate of points allowed per 100 possessions using adjusted formulas. |
| `DEF_RATING` | Defensive Rating: Points allowed per 100 possessions while the player is on the court. |
| `sp_work_DEF_RATING` | Internal variant of DEF_RATING based on tracking data. |
| `E_NET_RATING` | Estimated Net Rating: The difference between `E_OFF_RATING` and `E_DEF_RATING`. |
| `NET_RATING` | Net Rating: The difference between `OFF_RATING` and `DEF_RATING`. Higher = better. |
| `sp_work_NET_RATING` | Internal net rating calculation using `sp_work_` stats. |
| `AST_PCT` | Assist Percentage: % of teammate field goals assisted by the player while on the court. |
| `AST_TO` | Assist-to-Turnover Ratio: Assists divided by turnovers (`AST / TOV`). |
| `AST_RATIO` | Assist Ratio: Assists per 100 possessions. |
| `OREB_PCT` | Offensive Rebound Percentage: % of available offensive rebounds grabbed. |
| `DREB_PCT` | Defensive Rebound Percentage: % of available defensive rebounds grabbed. |
| `REB_PCT` | Total Rebound Percentage: % of available rebounds (offensive + defensive) grabbed. |
| `TM_TOV_PCT` | Team Turnover Percentage: Estimate of turnovers committed per 100 team possessions while the player is on the court. |
| `E_TOV_PCT` | Estimated Turnover Percentage: Estimate of player’s turnover rate using adjusted methods. |
| `EFG_PCT` | Effective Field Goal Percentage: Adjusts FG% to account for 3-point shots being worth more (`(FGM + 0.5 * 3PM) / FGA`). |
| `TS_PCT` | True Shooting Percentage: Shooting efficiency including FG, 3PT, and FT (`PTS / (2 * (FGA + 0.44 * FTA))`). |
| `USG_PCT` | Usage Percentage: % of team plays used by the player while on court. | Basically, How often is this guy finishing the play when he’s out there?”
| `E_USG_PCT` | Estimated Usage Percentage: Adjusted version of USG_PCT, possibly using tracking data. |
| `E_PACE` | Estimated Pace: Estimate of possessions per 48 minutes, team-adjusted. |
| `PACE` | Team Pace: Estimate of possessions per 48 minutes while the player is on the court. |
| `PACE_PER40` | Projected possessions per 40 minutes, used for comparative purposes. |
| `sp_work_PACE` | Internal pace metric, often based on player tracking systems. |
| `PIE` | Player Impact Estimate: NBA's holistic player impact metric showing overall contribution to team success. |
| `POSS` | Estimated number of possessions the player was on the floor for. Estimated Possessions = FGA + (0.44 × FTA) - ORB + TO  Possessions measure how often a team or player is involved in plays| 
| `FGM_PG` | Field Goals Made per Game. |
| `FGA_PG` | Field Goals Attempted per Game. |
| `_RANK` Columns (e.g., `AST_PCT_RANK`) | Rank of the player in that metric compared to other players in the dataset. |

| Feature Name      | Description                                                                 |
|-------------------|-----------------------------------------------------------------------------|
| `HEIGHT`          | Player's height, usually in feet-inches format (e.g., "6-7").                |
| `WEIGHT`          | Player's weight in pounds.                                                   |
| `POSITION`        | Player’s official NBA position (e.g., "G", "F", "C", "G-F", etc.).           |
| `AGE`             | Age of the player during the season.                                         |
| `EXP`             | Years of NBA experience (0 = rookie).                                        |